# Face Recognition using VGG Face Descriptor

Source: https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

In [1]:
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
import numpy as np

/home/agusgun/anaconda3/envs/basic/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Initialize Model

In [2]:
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Convolution2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))
 
model.add(Convolution2D(4096, (7, 7), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(4096, (1, 1), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(2622, (1, 1)))
model.add(Flatten())
model.add(Activation('softmax'))

In [3]:
model.load_weights('model_data/vgg_face_weights.h5')

In [4]:
vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

## Feature Extraction

In [5]:
def preprocess_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [6]:
%%time
img_representation = vgg_face_descriptor.predict(preprocess_image('../face_data/1_0.png'))[0]

CPU times: user 2.78 s, sys: 427 ms, total: 3.21 s
Wall time: 2.3 s


In [7]:
img_representation.shape

(2622,)

## Testing Using Own Dataset

In [8]:
import os

In [9]:
img_path_list = os.listdir('../face_data')
img_path_list = [os.path.join('../face_data', img_path) for img_path in img_path_list]

In [10]:
target = []
for img_path in img_path_list:
    if ('/1_' in img_path):
        target.append(1)
    elif ('/2_' in img_path):
        target.append(2)
    elif ('/7_' in img_path):
        target.append(3)
    elif ('/12_' in img_path):
        target.append(3)
    elif ('/17_' in img_path):
        target.append(4)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train_img_path_list, X_test_img_path_list, y_train, y_test = train_test_split(
    img_path_list, target, test_size=0.2, random_state=0
)

In [13]:
def extract_batch(img_path_list):
    result = []
    for img_path in img_path_list:
        result.append(vgg_face_descriptor.predict(preprocess_image(img_path))[0])
    return result

In [14]:
%%time
X_train_representation = extract_batch(X_train_img_path_list)
X_test_representation = extract_batch(X_test_img_path_list)

CPU times: user 14min 13s, sys: 24.3 s, total: 14min 37s
Wall time: 2min 7s


## Predict using Euclidean Distance

In [15]:
from scipy.spatial.distance import euclidean

In [16]:
def predict(image_representation_list, true_label, test_representation):
    minimum_label = None
    minimum_distance = np.inf
    
    for idx, image_representation in enumerate(image_representation_list):
        distance = euclidean(image_representation, test_representation)
        if distance < minimum_distance:
            minimum_distance = distance
            minimum_label = true_label[idx]
    
    return minimum_label

In [17]:
predict(X_train_representation, y_train, X_test_representation[0])

4

In [18]:
predicted_list = []
for test_representation in X_test_representation:
    predicted_list.append(predict(X_train_representation, y_train, test_representation))

In [19]:
from sklearn.metrics import accuracy_score, f1_score

In [20]:
print(accuracy_score(predicted_list, y_test))
print(f1_score(predicted_list, y_test, average='weighted'))

0.9824561403508771
0.982995951417004


# Testing using Downloaded Dataset